# Выпускной проект

# Проект "SQL. База данных книг"

1. [Описание проекта](#intro)
1. [Загрузка и изучение баз данных](#upload)
2. [Формирование запросов](#query)    
    2.1. [Запрос №1](#1)    
    2.2. [Запрос №2](#2)    
    2.3. [Запрос №3](#3)    
    2.4. [Запрос №4](#4)    
    2.5. [Запрос №5](#5)
3. [Выводы](#conclusion)

<a id="intro"></a>

## Описание проекта

Имеется крупный сервис для чтения книг по подписке. Информация о книгах собрана в несколько баз данных. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель проекта:** проанализировать базу данных путем создания SQL-запросов 

**Задачи:**
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так можно исключить из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками);
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных**

1. Таблица books. Содержит данные о книгах:
    - book_id — идентификатор книги;
    - author_id — идентификатор автора;
    - title — название книги;
    - num_pages — количество страниц;
    - publication_date — дата публикации книги;
    - publisher_id — идентификатор издателя.

2. Таблица authors. Содержит данные об авторах:
    - author_id — идентификатор автора;
    - author — имя автора.
    
3. Таблица publishers. Содержит данные об издательствах:
    - publisher_id — идентификатор издательства;
    - publisher — название издательства;
    
4. Таблица ratings. Содержит данные о пользовательских оценках книг:
    - rating_id — идентификатор оценки;
    - book_id — идентификатор книги;
    - username — имя пользователя, оставившего оценку;
    - rating — оценка книги.
    
5. Таблица reviews. Содержит данные о пользовательских обзорах:
    - review_id — идентификатор обзора;
    - book_id — идентификатор книги;
    - username — имя автора обзора;
    - text — текст обзора.
    
**Инструментарий:**

1. Python
    - *библиотеки:* pandas, sqlalchemy
    - *работа с данными:* read_csv, info, try-except, isna, sum, duplicated, set_option, create_engine, io.sql.read_sql
2. SQL
    - SELECT, FROM, WHERE, IN, COUNT, WITH, HAVING, GROUP BY, INNER JOIN, RIGHT JOIN, ORDER BY

<a id="upload"></a>

## Загрузка и изучение баз данных

Импорт библиотек и настройка отображения

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Настройка отображения значений с плавающей точкой
pd.set_option('display.float_format', '{:.2f}'.format)

Настройка подключения к базам данных SQL

In [2]:
db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp',
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432,
 'db': 'data-analyst-final-project-db'}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Загрузим базы данных

In [3]:
books = '''
        SELECT * FROM books
        '''

books = pd.io.sql.read_sql(books, con = engine)


authors = '''
        SELECT * FROM authors

        '''
authors = pd.io.sql.read_sql(authors, con = engine)


publishers =  '''
        SELECT * FROM publishers

        '''
publishers = pd.io.sql.read_sql(publishers, con = engine)


ratings =  '''
        SELECT * FROM ratings

        '''
ratings = pd.io.sql.read_sql(ratings, con = engine)


reviews =  '''
        SELECT * FROM reviews

        '''
reviews = pd.io.sql.read_sql(reviews, con = engine)

Изучим базы данных

In [4]:
# Определим функцию для знакомства с базой данных и проверки её на явные дубликаты
def database_check(db):
    print('Случайные 5 строк базы данных')
    display(db.sample(5, random_state=0))
    print('*'*50)
    print('Общая информация о базе данных')
    db.info()
    print('*'*50)
    print('Пропущенных значений в строках:')
    print(db.isna().sum())
    print('*'*50)
    print('Явных дубликатов в базе данных:', db.duplicated().sum())
    print('*'*50)

In [5]:
print('База данных книг (books)')
database_check(books)

print('База данных авторов (authors)')
database_check(authors)

print('База данных извательств (publishers)')
database_check(publishers)

print('База данных пользовательских рейтингов (ratings)')
database_check(ratings)

print('База данных пользовательских обзоров (reviews)')
database_check(reviews)

База данных книг (books)
Случайные 5 строк базы данных


,book_id,author_id,title,num_pages,publication_date,publisher_id
993,994,408,World War Z: An Oral History of the Zombie War,342,2006-09-12,68
859,860,425,The Scarlet Letter,279,2003-02-27,212
298,299,236,Harry Potter and the Chamber of Secrets (Harry...,341,1999-06-02,24
553,554,67,Scott Pilgrim Volume 1: Scott Pilgrim's Preci...,168,2004-08-18,196
672,673,235,The Catcher in the Rye,277,2001-01-30,31


**************************************************
Общая информация о базе данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
**************************************************
Пропущенных значений в строках:
book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64
**************************************************
Явных дубликатов в базе данных: 0
**************************************************
База данных авторов (authors)


,author_id,author
453,454,Orson Scott Card/Piotr W. Cholewa
482,483,Primo Levi/Stuart J. Woolf/Philip Roth
516,517,Rohinton Mistry
542,543,Stephen King/Bernie Wrightson
85,86,Charles Dickens/Nicola Bradbury/Hablot Knight ...


**************************************************
Общая информация о базе данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
**************************************************
Пропущенных значений в строках:
author_id    0
author       0
dtype: int64
**************************************************
Явных дубликатов в базе данных: 0
**************************************************
База данных извательств (publishers)
Случайные 5 строк базы данных


...

**************************************************
Общая информация о базе данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
**************************************************
Пропущенных значений в строках:
publisher_id    0
publisher       0
dtype: int64
**************************************************
Явных дубликатов в базе данных: 0
**************************************************
База данных пользовательских рейтингов (ratings)
Случайные 5 строк базы данных


,rating_id,book_id,username,rating
5434,5435,838,debra43,4
4215,4216,677,edwardbrown,4
2232,2233,359,wknight,4
4583,4584,722,amymckenzie,4
4312,4313,692,jasongoodman,3


**************************************************
Общая информация о базе данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
**************************************************
Пропущенных значений в строках:
rating_id    0
book_id      0
username     0
rating       0
dtype: int64
**************************************************
Явных дубликатов в базе данных: 0
**************************************************
База данных пользовательских обзоров (reviews)
Случайные 5 строк базы данных


,review_id,book_id,username,text
1533,1532,556,roberto26,People floor assume like new knowledge yet. Re...
2392,2392,860,fharris,Walk would relate worker two because financial...
502,788,292,nicoleburns,Bag early hotel. Foot fly quality.
1515,1514,549,ryanfranco,Hit hotel meet through cause standard behind. ...
1858,1857,674,edwardbrown,Change forward during current. Writer ready po...


**************************************************
Общая информация о базе данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
**************************************************
Пропущенных значений в строках:
review_id    0
book_id      0
username     0
text         0
dtype: int64
**************************************************
Явных дубликатов в базе данных: 0
**************************************************


**Вывод**

Информация представлена в 5 базах данных. Пропущенных и дублирующихся занчений нет, все данные представлены в нужном формате. Предобработка данных не требуется

<a id="query"></a>

## Формирование запросов

<a id="1"></a>

### Зарпос №1

Сколько книг вышло после 1 января 2000 года

In [6]:
query_books =   '''
                    SELECT
                        COUNT(book_id)
                    FROM
                        books
                    WHERE  
                        publication_date::date > '2000-01-01'
                    ''' 
query_books = pd.io.sql.read_sql(query_books, con = engine)  

print(f'После 1 января 2000 года вышло {query_books["count"][0]} книг')

После 1 января 2000 года вышло 819 книг


<a id="2"></a>

### Зарпос №2

Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
query_reviews = '''
            WITH book_reviews AS
                    (SELECT books.book_id AS book_id, 
                    COUNT(reviews.text) AS review_count,
                    books.title
                FROM reviews
                RIGHT JOIN books on reviews.book_id = books.book_id
                GROUP BY books.book_id),
                book_rating AS
                    (SELECT books.book_id AS book_id, 
                    AVG(ratings.rating) AS avg_rating,
                    books.title
                FROM ratings
                RIGHT JOIN books on ratings.book_id = books.book_id
                GROUP BY books.book_id)            
            SELECT book_rating.book_id AS book_id,
                    book_rating.title AS title,
                    book_reviews.review_count AS review_count,
                    book_rating.avg_rating AS avg_rating                    
            FROM book_reviews
            INNER JOIN book_rating ON book_reviews.book_id = book_rating.book_id
                    ''' 

query_reviews = pd.io.sql.read_sql(query_reviews, con = engine)    

print('Количество обзоров и средняя оценка каждой книги')
query_reviews

Количество обзоров и средняя оценка каждой книги


,book_id,title,review_count,avg_rating
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,997,Xenocide (Ender's Saga #3),3,3.40
997,998,Year of Wonders,4,3.20
998,999,You Suck (A Love Story #2),2,4.50


<a id="3"></a>

### Зарпос №3

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [10]:
query_publishers = '''
                        SELECT
                            publishers.publisher_id AS publisher_id,
                            publishers.publisher AS publisher_name,
                            COUNT(DISTINCT(books.book_id)) AS count_books
                        FROM
                            publishers
                        INNER JOIN books ON books.publisher_id = publishers.publisher_id
                        WHERE
                            books.num_pages > 50
                        GROUP BY
                            publishers.publisher_id, publishers.publisher
                        ORDER BY
                            count_books DESC


                        '''
query_publishers = pd.io.sql.read_sql(query_publishers, con = engine)


print(f'Издательство {query_publishers["publisher_name"][0]} выпустило больше всего книг - {query_publishers["count_books"][0]}')

Издательство Penguin Books выпустило больше всего книг - 42


<a id="4"></a>

### Зарпос №4

Определим автора с самой высокой средней оценкой книг (при учете только книг с 50 и более оценками)

In [11]:
query_authors = '''
                    SELECT
                        rating_books.author_id AS author_id,
                        rating_books.author_name AS author_name,
                        AVG(rating_books.avg_rating) AS author_avg_rating
                    FROM
                        (SELECT    
                            books.author_id AS author_id,
                            authors.author AS author_name,
                            ratings.book_id AS book_id,
                            AVG(ratings.rating) AS avg_rating    
                        FROM
                            ratings
                        INNER JOIN
                            books ON books.book_id = ratings.book_id
                        INNER JOIN
                            authors ON authors.author_id = books.author_id    
                        GROUP BY
                            ratings.book_id, books.author_id, authors.author
                        HAVING
                            COUNT(ratings.rating_id) > 49) AS rating_books
                    GROUP BY
                        rating_books.author_id, rating_books.author_name
                    ORDER BY
                        author_avg_rating DESC

                    '''
query_authors = pd.io.sql.read_sql(query_authors, con = engine)      

print(f'Среди авторов с высокорейтинговыми книгами {query_authors["author_name"][0]} имеют самый высокий средний рейтинг - {query_authors["author_avg_rating"][0]:.6f}')

Среди авторов с высокорейтинговыми книгами J.K. Rowling/Mary GrandPré имеют самый высокий средний рейтинг - 4.283844


<a id="5"></a>

### Зарпос №5

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
query_avg_reviews = '''
                        SELECT 
                            AVG(user_reviews.review_number) AS avg_review_number
                        FROM
                            (SELECT 
                                username AS username,
                                COUNT(review_id) AS review_number
                             FROM 
                                reviews
                             WHERE 
                                username IN
                                     (SELECT 
                                         username AS username
                                      FROM 
                                         ratings
                                      GROUP BY 
                                         username
                                      HAVING 
                                          COUNT(rating_id) > 50)
                             GROUP BY 
                                 username
                             ) AS user_reviews

                            '''

query_avg_reviews = pd.io.sql.read_sql(query_avg_reviews, con=engine)

print(f'Среднее количество обзоров от пользователей, которые поставили больше 50 оценок -  {query_avg_reviews["avg_review_number"][0]:.0f} штуки')

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок -  24 штуки


<a id="conclusion"></a>

## Выводы

- После 1 января 2020 года вышло 819 книг
- Книги в базе данных имеют от 0 до 8 отзывов и рейтинг от 1.5 до 5 пунктов
- Больше всего книг (42) выпустило издательство Penguin Books
- Самый высокий средний рейтинг (4.28) имеют два автора: Дж.К.Роулинг и Мери ГранПре
- Среднее количество отзывов от пользователей, которые поставили больше 50 оценок -  24